In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time

# Configure Chrome options
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)  # Keep browser open for debugging

# Initialize driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

try:
    print("1. Loading website...")
    driver.get("https://in.puma.com/in/en")
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    print(" Page loaded")
    
    # Strategy 1: Try clicking search button normally
    print("2. Attempting standard search...")
    try:
        search_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-test-id='search-button-nav']")))
        search_btn.click()
        
        # Wait for search input with multiple selectors
        input_selectors = [
            "input[placeholder='Search for products']",
            "input.search-input",
            "input[type='search']",
            "#search-input",
            ".search-bar input"
        ]
        
        search_input = None
        for selector in input_selectors:
            try:
                search_input = WebDriverWait(driver, 5).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, selector)))
                break
            except:
                continue
                
        if not search_input:
            raise Exception("Could not find search input with any selector")
            
        search_input.send_keys("shoes" + Keys.RETURN)
        print(" Standard search completed")
        
    except Exception as e:
        print(f" Standard search failed: {str(e)}")
        print("3. Falling back to direct URL approach...")
        
        # Strategy 2: Go directly to search results URL
        driver.get("https://in.puma.com/in/en/search?q=shoes")
        print(" Direct search URL loaded")

    # Wait for results with multiple possible selectors
    print("4. Waiting for search results...")
    product_selectors = [
        "[data-test-id='product-list-item']",
        ".product-grid_item",
        ".grid-item",
        ".product-card",
        ".tile-item"
    ]
    
    products = []
    for selector in product_selectors:
        try:
            products = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, selector)))
            if products:
                print(f" Found {len(products)} products using selector: {selector}")
                break
        except:
            continue
            
    if not products:
        raise Exception("Could not find any products with all selectors")
    
    # Continue with product interaction...
    products[0].click()
    print("5. Product page loaded")

except Exception as e:
    print(f" Final error: {str(e)}")
    # Save debugging information
    driver.save_screenshot("error.png")
    with open("page_source.html", "w", encoding="utf-8") as f:
        f.write(driver.page_source)
    print(" Saved screenshot and page source for debugging")

finally:
    input("Press Enter to close browser...")
    driver.quit()
    print(" Browser closed")

1. Loading website...
 Page loaded
2. Attempting standard search...
 Standard search completed
4. Waiting for search results...
 Found 24 products using selector: [data-test-id='product-list-item']
 Final error: Message: element click intercepted: Element <li data-test-id="product-list-item" data-product-id="312090_01">...</li> is not clickable at point (208, 607). Other element would receive the click: <div class="bg-puma-black-100 py-5 mobile:px-4 px-8 w-full">...</div>
  (Session info: chrome=135.0.7049.86)
Stacktrace:
	GetHandleVerifier [0x00E98073+60707]
	GetHandleVerifier [0x00E980B4+60772]
	(No symbol) [0x00CC0683]
	(No symbol) [0x00D0E940]
	(No symbol) [0x00D0CCE4]
	(No symbol) [0x00D0A887]
	(No symbol) [0x00D09B8B]
	(No symbol) [0x00CFE215]
	(No symbol) [0x00D2D04C]
	(No symbol) [0x00CFDCA4]
	(No symbol) [0x00D2D2C4]
	(No symbol) [0x00D4E824]
	(No symbol) [0x00D2CE46]
	(No symbol) [0x00CFC5D3]
	(No symbol) [0x00CFD424]
	GetHandleVerifier [0x010DBB53+2435075]
	GetHandleVerifier

Press Enter to close browser... 


 Browser closed


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import os

class PumaIndiaTester:
    def __init__(self):
        # Setup Chrome with enhanced options
        self.options = webdriver.ChromeOptions()
        self.options.add_argument("--start-maximized")
        self.options.add_argument("--disable-notifications")
        self.options.add_argument("--disable-popup-blocking")
        self.options.add_experimental_option("excludeSwitches", ["enable-automation"])
        self.options.add_experimental_option("detach", True)
        
        # Initialize driver
        self.service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=self.service, options=self.options)
        self.driver.implicitly_wait(5)
        
        # Create test directories
        os.makedirs("test_results/screenshots", exist_ok=True)
        os.makedirs("test_results/page_sources", exist_ok=True)

    def save_debug_info(self, prefix):
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        self.driver.save_screenshot(f"test_results/screenshots/{prefix}_{timestamp}.png")
        with open(f"test_results/page_sources/{prefix}_{timestamp}.html", "w", encoding="utf-8") as f:
            f.write(self.driver.page_source)

    def close_popups(self):
        popup_selectors = [
            "#cookie-banner-close-btn",
            ".modal-close",
            "[aria-label='Close']",
            ".newsletter-close"
        ]
        for selector in popup_selectors:
            try:
                WebDriverWait(self.driver, 3).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, selector))).click()
                print(f"🚫 Closed popup: {selector}")
                time.sleep(1)
            except:
                continue

    def run_tests(self):
        try:
            print("\n=== Starting PUMA India Test Suite ===")
            
            # Test 1: Homepage
            self.test_homepage()
            
            # Test 2: Search
            if not self.test_search_ui():
                print("🔄 Falling back to direct URL search")
                self.test_search_direct_url()
                
            # Test 3: Product Details
            self.test_product_details()
            
            # Test 4: Cart
            self.test_cart()
            
            print("\n✅✅ All tests completed successfully ✅✅")
            
        except Exception as e:
            print(f"\n❌❌ Test failed: {str(e)}")
            self.save_debug_info("final_error")
        finally:
            self.driver.quit()

    def test_homepage(self):
        print("\n1. Testing homepage...")
        self.driver.get("https://in.puma.com/in/en")
        self.close_popups()
        assert "PUMA" in self.driver.title
        self.save_debug_info("homepage")
        print("✅ Homepage loaded successfully")

    def test_search_ui(self):
        print("\n2. Testing UI search...")
        try:
            # Try clicking search button with multiple selectors
            search_selectors = [
                "button[data-test-id='search-button-nav']",
                ".search-icon",
                "#search-toggle",
                "[aria-label='Search']"
            ]
            
            clicked = False
            for selector in search_selectors:
                try:
                    search_btn = WebDriverWait(self.driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
                    search_btn.click()
                    clicked = True
                    print(f"🔍 Clicked search using: {selector}")
                    break
                except:
                    continue
            
            if not clicked:
                raise Exception("Could not find clickable search button")
            
            time.sleep(1)  # Wait for search panel animation
            
            # Try finding search input with multiple selectors
            input_selectors = [
                "input[placeholder='Search for products']",
                "input.search-input",
                "input[type='search']",
                "#search-input",
                ".search-bar input"
            ]
            
            search_input = None
            for selector in input_selectors:
                try:
                    search_input = WebDriverWait(self.driver, 5).until(
                        EC.visibility_of_element_located((By.CSS_SELECTOR, selector)))
                    print(f"⌨️ Found search input using: {selector}")
                    break
                except:
                    continue
            
            if not search_input:
                raise Exception("Could not find search input field")
            
            search_input.send_keys("shoes" + Keys.RETURN)
            time.sleep(2)  # Wait for search results
            
            # Verify results
            if not self.verify_search_results():
                raise Exception("No search results found")
            
            print("✅ UI search completed successfully")
            return True
            
        except Exception as e:
            print(f"⚠️ UI search failed: {str(e)}")
            self.save_debug_info("ui_search_fail")
            return False

    def test_search_direct_url(self):
        print("\n2a. Testing direct URL search...")
        self.driver.get("https://in.puma.com/in/en/search?q=shoes")
        self.close_popups()
        
        if not self.verify_search_results():
            raise Exception("No results found with direct URL search")
        
        print("✅ Direct URL search completed successfully")

    def verify_search_results(self):
        """Helper method to verify search results"""
        product_selectors = [
            "[data-test-id='product-list-item']",
            ".product-grid_item",
            ".grid-item",
            ".product-card",
            ".tile-item"
        ]
        
        for selector in product_selectors:
            try:
                products = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, selector)))
                if len(products) > 0:
                    print(f"🛍️ Found {len(products)} products using: {selector}")
                    self.save_debug_info("search_results")
                    return True
            except:
                continue
        
        return False

    def test_product_details(self):
        print("\n3. Testing product details...")
        # Use direct product URL
        product_url = "https://in.puma.com/in/en/pd/rs-x-bold-unisex-sneakers/372722.html"
        self.driver.get(product_url)
        self.close_popups()
        
        # Debug: Print current URL to verify correct page
        print(f"🌐 Current URL: {self.driver.current_url}")
        
        # Take screenshot before looking for elements
        self.save_debug_info("product_page_before")

        # Enhanced product title verification
        title_selectors = [
            "[data-test-id='product-title']",
            "h1.product-name",
            ".product-title",
            "h1.title",
            "h1[itemprop='name']",
            "h1.name"
        ]
        
        title = None
        title_found = False
        
        for selector in title_selectors:
            try:
                title = WebDriverWait(self.driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, selector)))
                if title.text.strip():
                    print(f"🏷️ Found product title using: {selector}")
                    print(f"📝 Title text: '{title.text}'")
                    title_found = True
                    break
            except Exception as e:
                print(f"⚠️ Not found with {selector}: {str(e)}")
                continue
        
        if not title_found:
            # Final attempt - search for any h1 tag
            try:
                h1_tags = self.driver.find_elements(By.TAG_NAME, "h1")
                for h1 in h1_tags:
                    if h1.text.strip():
                        title = h1
                        print(f"ℹ️ Found h1 tag with text: '{h1.text}'")
                        title_found = True
                        break
            except:
                pass
        
        if not title_found:
            self.save_debug_info("product_page_fail")
            raise Exception("Could not find product title with any selector")
        
        # Verify price
        price_selectors = [
            "[data-test-id='product-price']",
            ".product-price",
            ".price",
            ".price-final",
            "[itemprop='price']",
            ".amount",
            ".value"
        ]
        
        price_found = False
        for selector in price_selectors:
            try:
                price = self.driver.find_element(By.CSS_SELECTOR, selector)
                if price.text.strip():
                    print(f"💰 Found price using: {selector}")
                    print(f"💵 Price: {price.text}")
                    price_found = True
                    break
            except:
                continue
        
        if not price_found:
            print("⚠️ Could not find price element")
        
        # Verify product image
        image_selectors = [
            "[data-test-id='product-image']",
            ".product-image",
            ".main-image",
            "img[itemprop='image']",
            ".image-wrapper img"
        ]
        
        image_found = False
        for selector in image_selectors:
            try:
                image = self.driver.find_element(By.CSS_SELECTOR, selector)
                if image.get_attribute("src"):
                    print(f"🖼️ Found product image using: {selector}")
                    image_found = True
                    break
            except:
                continue
        
        if not image_found:
            print("⚠️ Could not find product image")
        
        self.save_debug_info("product_page_success")
        print("✅ Product details verified")

    def test_cart(self):
        print("\n4. Testing cart functionality...")
        # First ensure we're on a product page
        if "pd/" not in self.driver.current_url:
            self.driver.get("https://in.puma.com/in/en/pd/rs-x-bold-unisex-sneakers/372722.html")
            self.close_popups()
        
        # Select size if needed
        try:
            size_btn = WebDriverWait(self.driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-test-id='size-9']")))
            size_btn.click()
            time.sleep(1)
        except:
            print("ℹ️ No size selection needed")
        
        # Add to cart
        add_to_cart_selectors = [
            "[data-test-id='add-to-cart-button']",
            "#add-to-cart",
            ".add-to-cart-btn"
        ]
        
        added = False
        for selector in add_to_cart_selectors:
            try:
                add_btn = WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
                add_btn.click()
                added = True
                print(f"🛒 Clicked add to cart using: {selector}")
                break
            except:
                continue
        
        if not added:
            raise Exception("Could not add product to cart")
        
        time.sleep(2)  # Wait for cart to update
        
        # Go to cart
        cart_selectors = [
            "[data-test-id='cart-button']",
            ".minicart",
            ".cart-icon"
        ]
        
        for selector in cart_selectors:
            try:
                cart_btn = WebDriverWait(self.driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
                cart_btn.click()
                break
            except:
                continue
        
        # Verify cart items
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "[data-test-id='cart-item']")))
            print("🛒 Cart contains items")
        except:
            raise Exception("Could not verify items in cart")
        
        self.save_debug_info("cart_page")
        print("✅ Cart functionality verified")

# Run the tests
if __name__ == "__main__":
    tester = PumaIndiaTester()
    tester.run_tests()


=== Starting PUMA India Test Suite ===

1. Testing homepage...
✅ Homepage loaded successfully

2. Testing UI search...
🔍 Clicked search using: button[data-test-id='search-button-nav']
⌨️ Found search input using: input[type='search']
🛍️ Found 24 products using: [data-test-id='product-list-item']
✅ UI search completed successfully

3. Testing product details...
